In [ ]:
# MailSense — AI Email Summarizer & Action Planner


## Developer Plan (where code lives)

- src/agent/agent_main.py  -> ADK agent entrypoint (instantiates agent, registers tools, handles input)
- src/tools.py            -> Contains implementations for summarize_email, extract_actions, create_task, schedule_event, compose_reply, log_action
- src/memory.py           -> Read/write memory.json and helper functions
- src/planner.py          -> Implements planner rules (see src/planner_rules.md)
- src/utils.py            -> Utility functions: date parsing, similarity checking
- data/examples_emails.json -> Example inputs used for demonstration
- tasks.json, calendar.json, memory.json -> Persistent stores produced by the tools


In [ ]:
**Short pitch:** MailSense ingests email text, summarizes it, extracts actions (tasks, deadlines, contacts), recommends a next action (create task / schedule / reply template), and can execute a mock task creation. Demonstrates Tool Use, Memory, and Planning.

In [ ]:
**Demo GIF:** Demo GIF will go here (demo/demo.gif)

In [ ]:
**Concepts demonstrated:** 
1. Tool Use (registering and calling tools)
2. Memory (persistent storage of past tasks)
3. Planning / Multi-step reasoning (intent → extract → decide → act)

In [ ]:
**Example run:** (will show demonstration of loading an email, summarizing, extracting actions, and creating a task)

In [ ]:
**Reproducibility & run instructions:** See docs/methodology.md and README.md for exact environment & commands.

In [ ]:
**Results & Evaluation:** Placeholder for evaluation table and scoring of the sample emails.

In [ ]:
**Methodology:** See docs/methodology.md